In [20]:
# Logs
import datetime

time = datetime.datetime.today()
daytime = time.strftime("%Y%m%d%H%M")
logPath = "logs/" + "trainingLog" + daytime + ".txt"

log = open(logPath, "a+")

log.write("Start logging..." + daytime)

logs/trainingLog202311271526.txt


In [1]:
# Using CUDA
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"
print("Used GPU", os.environ["CUDA_VISIBLE_DEVICES"])
log.write("Used GPU", os.environ["CUDA_VISIBLE_DEVICES"])

Used GPU 1,2


In [2]:
# Test BERT
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
example_text = '餐廳11點開始沒問題，位置保留十分鐘也沒問題'
bert_input = tokenizer(example_text,padding='max_length',
                       max_length = 10,
                       truncation=True,
                       return_tensors="pt")
# ------- bert_input -----
print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

tensor([[ 101, 7623, 2453, 8111, 7953, 7274, 1993, 3760, 1558,  102]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [3]:
import torch
import numpy as np
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
labels = {'0':0,
          '1':1
          }
value_to_labels = {
    0:'bad',
    1:'good',
}

class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = [label for label in df['rate']]
        self.texts = [tokenizer(text, 
                                padding='max_length', 
                                max_length = 512, 
                                truncation=True,
                                return_tensors="pt") 
                      for text in df['comment']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

In [4]:
import pandas as pd

log.write("Importing csv data...")
comment_text_df = pd.read_csv('../database/commentData.csv')

# bbc_text_df.head()



df = pd.DataFrame(comment_text_df)

display(df)
log.write("splitting the dataset...")
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))
log.write("len of the dataframes (train, val, test): ", len(df_train),len(df_val), len(df_test))

,rate,comment
0,0,前幾日去家樂福，要出去停車場時，動線設計相當不流暢，開車出入危險，照明也昏暗，尤其接近中午時...
1,1,接手台糖量販店之後，無論是在商品齊備程度、提供的服務與設施、配合進駐廠商的完全性都遠勝從前台...
2,1,1.這間家樂福離家裡很近，隨時要補貨都超方便的 2.之前假日都會有一個大型機器人🤖️出現表...
3,0,使用外送平台點了一包鴻禧菇，結果送來這種快要壞掉發臭的菇，根本就不能吃，東西壞了能不能就別拿...
4,1,是我目前逛過全高雄市最好的家樂福、開車🚗好停車還不用收費、顧客水準品質普遍比較高、大多為小家...
...,...,...
5141,0,外觀有感。
5142,1,風景很好
5143,1,風景優美
5144,1,很棒的體驗


4116 515 515


In [5]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

In [6]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):
    log.write("Training...")
  # 通过Dataset类获取训练和验证集
    train, val = Dataset(train_data), Dataset(val_data)
    # DataLoader根据batch_size获取数据，训练时选择打乱样本
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)
  # 判断是否使用GPU
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
            log.write("Using CUDA!")
            model = model.cuda()
            criterion = criterion.cuda()
    # 开始进入训练循环
    for epoch_num in range(epochs):
      # 定义两个变量，用于存储训练集的准确率和损失
            total_acc_train = 0
            total_loss_train = 0
      # 进度条函数tqdm
            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
        # 通过模型得到输出
                output = model(input_id, mask)
                # 计算损失
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                # 计算精度
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
        # 模型更新
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            # ------ 验证模型 -----------
            # 定义两个变量，用于存储验证集的准确率和损失
            total_acc_val = 0
            total_loss_val = 0
      # 不需要计算梯度
            with torch.no_grad():
                # 循环获取数据集，并用训练好的模型进行验证
                for val_input, val_label in val_dataloader:
          # 如果有GPU，则使用GPU，接下来的操作同训练
                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)
  
                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'''Epochs: {epoch_num + 1} 
              | Train Loss: {total_loss_train / len(train_data): .3f} 
              | Train Accuracy: {total_acc_train / len(train_data): .3f} 
              | Val Loss: {total_loss_val / len(val_data): .3f} 
              | Val Accuracy: {total_acc_val / len(val_data): .3f}''')
            log.write(
                f'''Epochs: {epoch_num + 1}
              | Train Loss: {total_loss_train / len(train_data): .3f}
              | Train Accuracy: {total_acc_train / len(train_data): .3f}
              | Val Loss: {total_loss_val / len(val_data): .3f}
              | Val Accuracy: {total_acc_val / len(val_data): .3f}''')

In [7]:
EPOCHS = 10
model = BertClassifier()
model = nn.DataParallel(model)
LR = 1e-6
log.write("Epoch:", EPOCHS)
log.write(("LR:", LR))
train(model, df_train, df_val, LR, EPOCHS)

  0%|          | 5/2058 [00:12<1:22:50,  2.42s/it]


KeyboardInterrupt: 

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():
        for test_input, test_label in test_dataloader:
              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)
              output = model(input_id, mask)
              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc   
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)

In [17]:
day = time.strftime('%Y%m%d')
modelPath = "models/state_model_" + day + ".pt"
log.write("Saving the trained model...")
torch.save(model.state_dict(), modelPath)
log.write("Model has saved in models folder, name is " + modelPath)

models/state_model_20231127.pt


In [ ]:
log.write("Cleaning cache in GPU memory...")
torch.cuda.empty_cache()

In [ ]:
log.write("Done!")
log.close()